# Trade Visualizer for Google Colab
This notebook visualizes stock trades by generating candlestick charts for a random selection of stocks. The program reads trade data from a CSV file, fetches historical stock data using Yahoo Finance, and plots charts with marked buy and sell trades.

Ensure you upload your CSV file when prompted to proceed.

In [ ]:
# Import required libraries
import plotly.graph_objects as go
import pandas as pd
import yfinance as yf
import random

## Step 1: Load Example CSV File
In this example, we will load the 'trades.csv' file containing sample trade data. The file should have columns including "Sym" (symbol), "Time" (trade time), "Price" (trade price), and "Bot/Sld" (buy or sell indicator).

In [ ]:
# Load the CSV file
data = pd.read_csv('trades.csv', parse_dates=['Time'])

## Step 2: Random Selection of Stocks
The following code will select a random sample of 10 stocks from the data you uploaded.

In [ ]:
# Get a random sample of 10 unique symbols
random_symbols = random.sample(list(data['Sym'].unique()), min(10, len(data['Sym'].unique())))

## Step 3: Plot Candlestick Charts
The following code will generate candlestick charts for each of the randomly selected stocks. Buy and sell trades are marked on the charts.

This step might take some time, depending on the amount of data and the number of symbols selected.

In [ ]:
# Iterate through each randomly selected symbol in the dataset
for symbol in random_symbols:
    # Filter trades for the current symbol
    symbol_trades = data[data['Sym'] == symbol]
    
    # Get the earliest and latest trade dates for this symbol
    min_trade_date = symbol_trades['Time'].min()
    max_trade_date = symbol_trades['Time'].max()

    # Define the start and end date for fetching historical data
    start_date = pd.to_datetime(min_trade_date - pd.DateOffset(months=1)).date()
    end_date = pd.to_datetime(max_trade_date + pd.DateOffset(months=1)).date()

    # Fetch historical data for the stock using yfinance
    historical_data = yf.download(symbol, start=start_date, end=end_date)

    # Ensure historical data is available
    if not historical_data.empty:
        # Filter out non-trading dates to avoid gaps
        historical_data.index = pd.to_datetime(historical_data.index).date

        # Plot the candlestick chart using Plotly
        fig = go.Figure(data=[go.Candlestick(
            x=historical_data.index,
            open=historical_data['Open'],
            high=historical_data['High'],
            low=historical_data['Low'],
            close=historical_data['Close'],
            name='Candlesticks'
        )])

        # Adding buy and sell annotations for all trades of the same symbol
        y_offset = 0.02  # Offset to avoid overlapping markers
        for idx, row in symbol_trades.iterrows():
            # Convert the trade time to datetime.date for consistency
            trade_time = pd.to_datetime(row['Time']).date()
            trade_price = row['Price']
            trade_type = row['Bot/Sld']

            # Make sure the trade price is within a reasonable range to avoid markers being far from the actual range
            min_price = historical_data['Low'].min()
            max_price = historical_data['High'].max()

            if min_price <= trade_price <= max_price and start_date <= trade_time <= end_date:
                # Apply a small offset only if it doesn't push the marker outside of reasonable range
                adjusted_price = trade_price + (y_offset if trade_type == 'Bot' else -y_offset)

                # Ensure adjusted price is within bounds
                if adjusted_price < min_price:
                    adjusted_price = min_price
                elif adjusted_price > max_price:
                    adjusted_price = max_price

                # Add trade markers
                fig.add_trace(go.Scatter(
                    x=[trade_time],
                    y=[adjusted_price],
                    mode='markers+text',
                    marker=dict(
                        color='green' if trade_type == 'Bot' else 'red',
                        size=10,
                        symbol='triangle-up' if trade_type == 'Bot' else 'triangle-down'
                    ),
                    text=['Buy' if trade_type == 'Bot' else 'Sell'],
                    textposition='top center',
                    name='Entry/Exit'
                ))

        # Update layout to include volume and format chart without gaps
        fig.update_layout(
            title=f'{symbol} - Daily Candlestick Chart with Trades (No Gaps)',
            xaxis_title='Date',
            yaxis_title='Price',
            xaxis_rangeslider_visible=False,
            xaxis=dict(
                type='category',
                tickangle=-45,  # Rotate the labels for better readability
                tickformat='%b %d',  # Format the date labels to show only month and day
                tickmode='auto',  # Automatically adjust the tick spacing
                nticks=15  # Limit the number of ticks to prevent overcrowding
            ),
            template='plotly_dark'
        )

        # Show the chart for the current symbol
        fig.show()
    else:
        print(f"No historical data found for symbol {symbol} between {start_date} and {end_date}.")